**Импорт Пакетов функций**



In [1]:
#@title
import numpy as np
import math
import cmath
from functools import reduce
from copy import deepcopy
import matplotlib.pyplot as plt
import numba as nb

**Функции квантовых гейтов**

In [26]:
#@title
H = np.array([[1/math.sqrt(2),1/math.sqrt(2)],[1/math.sqrt(2),-1/math.sqrt(2)]], dtype = 'complex_')
X = np.array([[0,1],[1,0]], dtype = 'complex_')
I = np.array([[1,0],[0,1]], dtype = 'complex_')
T = np.array([[1,0],[0, cmath.exp((1j*cmath.pi)/4)]], dtype = 'complex_')
T_ermit = np.array([[1,0],[0, cmath.exp((-1j*cmath.pi)/4)]], dtype = 'complex_')
V = np.array([[1,0,0,0],[0,1,0,0],[0,0,(1+1j)/2,(1-1j)/2],[0,0,(1-1j)/2,(1+1j)/2]], dtype = 'complex_')
V_ermit = np.array([[1,0,0,0],[0,1,0,0],[0,0,(1-1j)/2,(1+1j)/2],[0,0,(1+1j)/2,(1-1j)/2]], dtype = 'complex_')
CNOT = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]], dtype = 'complex_')
Zgate = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]], dtype = 'complex_')
Toffoli = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
           [0, 1, 0, 0, 0, 0, 0, 0],
           [0, 0, 1, 0, 0, 0, 0, 0],
           [0, 0, 0, 1, 0, 0, 0, 0],
           [0, 0, 0, 0, 1, 0, 0, 0],
           [0, 0, 0, 0, 0, 1, 0, 0],
           [0, 0, 0, 0, 0, 0, 0, 1],
           [0, 0, 0, 0, 0, 0, 1, 0]],dtype = 'complex_')




def UnitGate(q, matrix, q0):
  qubit = deepcopy(q)
  for i in range(len(qubit)):
    if (i % (1<<(q0+1))) < (1<<(q0)):
      r = 0
    else:
      r = 1
    qubit[i] = q[i|(1<<q0)] * matrix[r][1] + q[(i|(1<<q0))^(1<<q0)] * matrix[r][0]
  return qubit

#if you need to impose some kind of control operation NOT ,
# then you need to write (-) and the desired number. If it is 0, you need to
# write the same number in place of the control qubit as the controlled one

def UnitGatetwo(q,matrix,q1,q0):
  qubit = deepcopy(q)
  if q1 >= 0 and q1 != q0:
    for i in range(len(q)):
      if (i % (1<<(q0+1))) < (1<<(q0)):
        r1 = 0
      else:
        r1 = 1
      if (i %(1<<(q1+1))) < (1<<(q1)):
        r2 = 0
      else:
        r2 = 1
      qubit[i] = q[(i|(1<<q0))|(1<<q1)] * matrix[2*r2 +r1][3]  \
      + q[((i|(1<<q0))|(1<<q1))^(1<<q0)] * matrix[2*r2 +r1][2] \
      + q[((i|(1<<q0))|(1<<q1))^(1<<q1)] * matrix[2*r2 +r1][1] \
      + q[(((i|(1<<q0)|(1<<q1))^(1<<q1))^(1<<q0))] * matrix[2*r2 +r1][0]
  elif q1 == q0:
    qubit = UnitGate(qubit,X,0)
    qubit = UnitGatetwo(qubit,matrix,0,q0)
    qubit = UnitGate(qubit,X,0)
  else:
    qubit = UnitGate(qubit,X,-q1)
    qubit = UnitGatetwo(qubit,matrix,-q1,q0)
    qubit = UnitGate(qubit,X,-q1)

  return qubit

def UnitGatethree(qubit, matrix, q2, q1, q0):
    q = deepcopy(qubit)
    if q2 >= 0 and q1 >=0 and q2 != q0 and q1 != q0:
      for i in range(len(q)):
          if (i % (1 << (q0 + 1))) < (1 << q0):
              r1 = 0
          else:
              r1 = 1
          if (i % (1 << (q1 + 1))) < (1 << q1):
              r2 = 0
          else:
              r2 = 1
          if (i % (1 << (q2 + 1))) < (1 << q2):
              r3 = 0
          else:
              r3 = 1

          q[i] = qubit[(i|(1<<q0))|(1<<q1)|(1<<q2)]*matrix[4*r3+2*r2+r1][7] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2))^(1<<q0)]*matrix[4*r3+2*r2+r1][6] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2))^(1<<q1)]*matrix[4*r3+2*r2+r1][5] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2))^(1<<q0)^(1<<q1)]\
          *matrix[4*r3+2*r2+r1][4] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2))^(1<<q2)]*matrix[4*r3+2*r2+r1][3] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2))^(1<<q0)^(1<<q2)]\
          *matrix[4*r3+2*r2+r1][2] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2))^(1<<q1)^(1<<q2)]\
          *matrix[4*r3+2*r2+r1][1] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2))^(1<<q1)^(1<<q2)^(1<<q0)]\
          *matrix[4*r3+2*r2+r1][0]
    elif q2 == q0:
      qubit = UnitGate(qubit,X,0)
      qubit = UnitGatethree(qubit,matrix,0,q1,q0)
      qubit = UnitGate(qubit,X,0)
    elif q1 == q0:
      qubit = UnitGate(qubit,X,0)
      qubit = UnitGatethree(qubit,matrix,q2,0,q0)
      qubit = UnitGate(qubit,X,0)
    elif q2 < 0:
      qubit = UnitGate(qubit,X,-q2)
      qubit = UnitGatethree(qubit,matrix, -q2,q1,q0)
      qubit = UnitGate(qubit,X,-q2)
    elif q1 < 0:
      qubit = UnitGate(qubit,X,-q1)
      qubit = UnitGatethree(qubit,matrix, q2, -q1,q0)
      qubit = UnitGate(qubit,X,-q1)
    return q

def UnitGatefour(qubit, matrix, q3, q2, q1, q0):
    q = deepcopy(qubit)
    if q3>=0 and q2>=0 and q1>=0 and q3!=q0 and q2!=q0 and q1!=q0:
      for i in range(len(q)):
          if (i % (1 << (q0 + 1))) < (1 << q0):
              r1 = 0
          else:
              r1 = 1
          if (i % (1 << (q1 + 1))) < (1 << q1):
              r2 = 0
          else:
              r2 = 1
          if (i % (1 << (q2 + 1))) < (1 << q2):
              r3 = 0
          else:
              r3 = 1
          if (i % (1 << (q3 + 1))) < (1 << q3):
              r4 = 0
          else:
              r4 = 1

          q[i] = qubit[(i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3)]*matrix[8*r4 + 4*r3+2*r2+r1][15] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q0)]*matrix[8*r4 + 4*r3+2*r2+r1][14] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q1)]*matrix[8*r4 + 4*r3+2*r2+r1][13] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q1)^(1<<q0)]*matrix[8*r4 + 4*r3+2*r2+r1][12] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q2)]*matrix[8*r4 + 4*r3+2*r2+r1][11] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q2)^(1<<q0)]*matrix[8*r4 + 4*r3+2*r2+r1][10] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q2)^(1<<q1)]*matrix[8*r4 + 4*r3+2*r2+r1][9] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q2)^(1<<q1)^(1<<q0)]*matrix[8*r4 + 4*r3+2*r2+r1][8]\
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q3)]*matrix[8*r4 + 4*r3+2*r2+r1][7] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q3)^(1<<q0)]*matrix[8*r4 + 4*r3+2*r2+r1][6] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q3)^(1<<q1)]*matrix[8*r4 + 4*r3+2*r2+r1][5] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q3)^(1<<q1)^(1<<q0)]*matrix[8*r4 + 4*r3+2*r2+r1][4]\
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q3)^(1<<q2)]*matrix[8*r4 + 4*r3+2*r2+r1][3] \
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q3)^(1<<q2)^(1<<q0)]*matrix[8*r4 + 4*r3+2*r2+r1][2]\
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q3)^(1<<q2)^(1<<q1)]*matrix[8*r4 + 4*r3+2*r2+r1][1]\
          + qubit[((i|(1<<q0))|(1<<q1)|(1<<q2)|(1<<q3))^(1<<q3)^(1<<q2)^(1<<q1)^(1<<q0)]*matrix[8*r4 + 4*r3+2*r2+r1][0]

    elif q3 == q0:
      qubit = UnitGate(qubit,X,0)
      qubit = UnitGatefour(qubit,matrix,0,q2,0,q0)
      qubit = UnitGate(qubit,X,0)
    elif q2 == q0:
      qubit = UnitGate(qubit,X,0)
      qubit = UnitGatefour(qubit,matrix,q3,0,q1,q0)
      qubit = UnitGate(qubit,X,0)
    elif q1 == q0:
      qubit = UnitGate(qubit,X,0)
      qubit = UnitGatefour(qubit,matrix,q3,q2,0,q0)
      qubit = UnitGate(qubit,X,0)
    elif q3 < 0:
      qubit = UnitGate(qubit,X,-q3)
      qubit = UnitGatefour(qubit,matrix, -q3,q2, q1,q0)
      qubit = UnitGate(qubit,X,-q3)
    elif q2 < 0:
      qubit = UnitGate(qubit,X,-q2)
      qubit = UnitGatefour(qubit,matrix, q3,-q2,q1,q0)
      qubit = UnitGate(qubit,X,-q2)
    elif q1 < 0:
      qubit = UnitGate(qubit,X,-q1)
      qubit = UnitGatefour(qubit,matrix, q3,q2, -q1,q0)
      qubit = UnitGate(qubit,X,-q1)

    return q




def Toffoli2():
  a = np.eye(16, dtype = 'complex_')
  a[15][15] = 0
  a[14][15] = 1
  a[15][14] = 1
  a[14][14] = 0
  return a

def V3():
  a = np.eye(16, dtype = 'complex_')
  a[15][15] = (1+1j)/2
  a[14][15] = (1-1j)/2
  a[15][14] = (1-1j)/2
  a[14][14] = (1+1j)/2
  return a



# Перевод i кубита по счету в битовую запись размера n
# Ввод (n) - количество кубит, (i) - номер кубита 
# вывод int запись бита
def inttobit(n, i):
  return list(map(int, list(format(i, "0{0}b".format(n)))))

def powcomplex(complexmatrix):
  return (complexmatrix * (complexmatrix.real - 1j*complexmatrix.imag)).real

In [ ]:
#@title
n = int(input())
q = np.zeros((2**(n)), dtype = 'complex_')
q[3] = 1
print(q)
UnitGatetwo(q,CNOT,1,1)

**Grover 's algorithm**

the sat problem

In [20]:
#@title
def oraclsatproblem(qubit,n,step, *args):
  for i in range(step):
    if len(args[i]) == 1:
      qubit = UnitGatetwo(qubit,CNOT,args[i][0], n+i)
    elif len(args[i]) == 2:
      qubit = UnitGatethree(qubit,Toffoli, args[i][0], args[i][1], n+i)
    elif len(args[i]) == 3:
      qubit = UnitGatefour(qubit,Toffoli2(), args[i][0], args[i][1], args[i][2], n+i)
    else:
      break
  if step == 1:
    qubit = UnitGatetwo(qubit,CNOT,n,n+1)
  elif step ==2:
    qubit = UnitGatethree(qubit,Toffoli,n,n+1,n+2)
  elif step == 3:
    qubit == UnitGatefour(qubit,Toffoli2(),n,n+1,n+2,n+3)
  elif step == 4:
    qubit = UnitGatetwo(qubit,V,n+3,n+4)
    qubit = UnitGatefour(qubit,Toffoli2(),n,n+1,n+2,n+3)
    qubit = UnitGatetwo(qubit,V_ermit,n+3,n+4)   
    qubit = UnitGatefour(qubit,Toffoli2(),n,n+1,n+2,n+3)
    qubit = UnitGatefour(qubit,V3(),n,n+1,n+2,n+4)
  for j in range(step):
    if len(args[j]) == 1:
      qubit = UnitGatetwo(qubit,CNOT,args[j][0], n+j)
    elif len(args[j]) == 2:
      qubit = UnitGatethree(qubit,Toffoli, args[j][0], args[j][1], n+j)
    elif len(args[j]) == 3:
      qubit = UnitGatefour(qubit,Toffoli2(), args[j][0], args[j][1], args[j][2], n+j)
    else:
      break
  return qubit


def diffuzer(qubit,n):
  if n == 2:
    qubit = UnitGatetwo(qubit,CNOT,1,1)
  if n == 3:
    qubit = UnitGatethree(qubit,Toffoli,2,-1,2)
  if n == 4:
    qubit = UnitGatefour(qubit,Toffoli2(),3,-1,-2,3)
  return qubit


def Groversat(qubit,n,step, *args):
  for i in range((int(math.log2(n)))-1):
    qubit = oraclsatproblem(qubit,n,step, *args)
    qubit = diffuzer(qubit,n)
  return qubit

In [ ]:
#@title
n = int(input())
step = int(input())
q = np.zeros((2**(n+step+1)), dtype = 'complex_')
q[0] = 1
q = UnitGate(q,X,n+step)
for i in range(n):
  q = UnitGate(q,H,i)
q = UnitGate(q,H,n+step)
list1 = [0,1]
list2 = [-1,2]
list3 = [0,1,2]
list4 = [2]
pow1 = Groversat(q,n,step, list1,list2,list3,list4)
pow1 = powcomplex(pow1)
pow1

In [ ]:
pow2 = np.where(pow1 > 0.01)
for i in range(len(pow2)):
  print(bin(pow2[i]))

if there is a boolean expression(it only works with a maximum of 3 conditions at the same time in one step)

In [8]:
#@title
def oraclgroverbool(qubit):
  step = int(input())
  for i in range(step):
    i = []
    for a in input().split():
      i.append( int(a) )
    if len(i) == 1:
      qubit = UnitGatetwo(qubit,CNOT,i[0], (int(math.log2(len(qubit)))-1))
    elif len(i) == 2:
      qubit = UnitGatethree(qubit,Toffoli,i[0],i[1], (int(math.log2(len(qubit)))-1))
    elif len(i) == 3:
      qubit = UnitGatefour(qubit,Toffoli2(),i[0],i[1],i[2], (int(math.log2(len(qubit)))-1))
    else:
      break
  return qubit

def diffuserbool(qubit):
  for i in range (int(math.log2(len(qubit)))-1):
    qubit = UnitGate(qubit,H,i)
    qubit = UnitGate(qubit,X,i)
  qubit = UnitGate(qubit,H,(int(math.log2(len(qubit))) - 2))
  if int(math.log2(len(qubit))) == 3:
    qubit = UnitGatetwo(qubit,CNOT,0, (int(math.log2(len(qubit)))-2))
  elif int(math.log2(len(qubit))) == 4:
    qubit = UnitGatethree(qubit,Toffoli,0,1, (int(math.log2(len(qubit)))-2))
  elif int(math.log2(len(qubit))) == 5:
    qubit = UnitGatefour(qubit,Toffoli2(),0,1,2, (int(math.log2(len(qubit)))-2))
  qubit = UnitGate(qubit,H,(int(math.log2(len(qubit))) - 2))
  for i in range (int(math.log2(len(qubit)))-1):
    qubit = UnitGate(qubit,X,i)
    qubit = UnitGate(qubit,H,i)
  return qubit

def Groverbool(q,n):
  for i in range (n+1):
    q = UnitGate(q,H,i)
  for j in range(max(int(round(math.sqrt(n)))-1, 1)):
    q = oraclgroverbool(q)
    q = diffuserbool(q)
  return q


In [ ]:
#@title
n = int(input())
q = np.zeros((2**(n+1)), dtype = 'complex_')
q[0] = 1
q = UnitGate(q,X,n)
q = Groverbool(q,n)
q = powcomplex(q)
indicator = 'yes'
if indicator =='yes':
  binstr = []
  for i in range(2**(n+1)):
    byt = bin(i)[2:].zfill(n+1)
    binstr.append(byt)
  plt.figure(figsize= (35,10), dpi= (300))
  plt.xticks(rotation=90)
  plt.bar(binstr, q)
